In [49]:
import pandas as pd
import math


def truth_table(n, ones):
    return [1 if i in ones else 0 for i in range(2**n)]


def check_P0(values, n):
    return values[0] == 0


def check_P1(values, n):
    return values[-1] == 1


def check_S(values, n):
    N = 2**n
    for i in range(N):
        if values[i] != 1 - values[N - 1 - i]:
            return False
    return True


def check_M(values, n):
    N = 2**n
    for i in range(N):
        for j in range(N):
            if (i & ~j) == 0 and values[i] > values[j]:
                return False
    return True


def pascal_triangle(values):
    rows = [values[:]]
    while len(rows[-1]) > 1:
        prev = rows[-1]
        nxt = [(prev[k] ^ prev[k + 1]) for k in range(len(prev) - 1)]
        rows.append(nxt)
    return rows


def check_L(values, n):
    rows = pascal_triangle(values)
    coeffs = [row[0] for row in rows]
    for idx, c in enumerate(coeffs):
        if c == 1 and (idx & (idx - 1)) != 0:
            return False
    return True


def classify_function(values):
    n = (len(values) - 1).bit_length()

    return {
        "P₀": check_P0(values, n),
        "P₁": check_P1(values, n),
        "S": check_S(values, n),
        "M": check_M(values, n),
        "L": check_L(values, n),
    }


def from_function_number(num):
    bits = list(map(int, bin(num)[2:]))
    N = len(bits)
    n = math.ceil(math.log2(N))

    if 2**n != N:
        bits = [0] * (2**n - N) + bits

    return bits


def menu():
    print("Выберите способ ввода функции:")
    print("1 - ввести единичные наборы (ones)")
    print("2 - ввести нулевые наборы (zeros)")
    print("3 - ввести номер функции (в десятичной форме)")
    choice = input("Ваш выбор: ")

    if choice == "1":
        ones = list(
            map(int, input("Введите номера единичных наборов через пробел: ").split())
        )
        n = max(ones).bit_length()
        values = truth_table(n, ones)
    elif choice == "2":
        zeros = list(
            map(int, input("Введите номера нулевых наборов через пробел: ").split())
        )
        n = max(zeros).bit_length()
        ones = [i for i in range(2**n) if i not in zeros]
        values = truth_table(n, ones)
    elif choice == "3":
        num = int(input("Введите номер функции (в десятичной форме): "))
        values = from_function_number(num)
    else:
        print("Неверный выбор.")
        return

    df = pd.DataFrame([classify_function(values)])
    df = df.style.hide(axis="index")
    display(df)


if __name__ == "__main__":
    menu()

Выберите способ ввода функции:
1 - ввести единичные наборы (ones)
2 - ввести нулевые наборы (zeros)
3 - ввести номер функции (в десятичной форме)


P₀,P₁,S,M,L
False,False,False,False,False


In [50]:
import math

SUBS = {
    "0": "₀", "1": "₁", "2": "₂", "3": "₃", "4": "₄",
    "5": "₅", "6": "₆", "7": "₇", "8": "₈", "9": "₉"
}

def subscript(num):
    return "".join(SUBS[d] for d in str(num))


def truth_table(n, ones):
    return [1 if i in ones else 0 for i in range(2**n)]


def from_function_number(num):
    bits = list(map(int, bin(num)[2:]))
    N = len(bits)
    n = math.ceil(math.log2(N))

    if 2**n != N:
        bits = [0] * (2**n - N) + bits

    return bits

def pascal_triangle(values):
    rows = [values[:]]
    while len(rows[-1]) > 1:
        prev = rows[-1]
        nxt = [(prev[k] ^ prev[k + 1]) for k in range(len(prev) - 1)]
        rows.append(nxt)
    return rows


def zhegalkin_polynomial(values):
    n = (len(values) - 1).bit_length()
    rows = pascal_triangle(values)

    coeffs = [row[0] for row in rows]
    terms = []

    for i, c in enumerate(coeffs):
        if c == 0:
            continue
        if i == 0:
            terms.append("1")
        else:
            vars = []
            for bit in range(n - 1, -1, -1):
                if (i >> bit) & 1:
                    vars.append(f"x{subscript(n - bit)}")
            terms.append("".join(vars))
    if not terms:
        return "0"
    return " ⊕ ".join(terms)

def menu():
    print("Выберите способ ввода функции:")
    print("1 - ввести единичные наборы (ones)")
    print("2 - ввести нулевые наборы (zeros)")
    print("3 - ввести номер функции (в десятичной форме)")
    choice = input("Ваш выбор: ")

    if choice == "1":
        ones = list(
            map(int, input("Введите номера единичных наборов через пробел: ").split())
        )
        n = max(ones).bit_length()
        values = truth_table(n, ones)
    elif choice == "2":
        zeros = list(
            map(int, input("Введите номера нулевых наборов через пробел: ").split())
        )
        n = max(zeros).bit_length()
        ones = [i for i in range(2**n) if i not in zeros]
        values = truth_table(n, ones)
    elif choice == "3":
        num = int(input("Введите номер функции (в десятичной форме): "))
        values = from_function_number(num)
    else:
        print("Неверный выбор.")
        return

    print("f =", zhegalkin_polynomial(values))


if __name__ == "__main__":
    menu()

Выберите способ ввода функции:
1 - ввести единичные наборы (ones)
2 - ввести нулевые наборы (zeros)
3 - ввести номер функции (в десятичной форме)
f = 1 ⊕ x₃ ⊕ x₂ ⊕ x₂x₃ ⊕ x₁x₂x₃


In [51]:
assert int("10001001", 2) == 137